In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [13]:
dates_dimension = pd.DataFrame(pd.read_csv("date_dimension.csv"))
weather_dimension = pd.DataFrame(pd.read_csv("weather_dimension.csv"))
mobility_dimension = pd.DataFrame(pd.read_csv("mobility_dimension.csv"))
special_measures_dimension = pd.DataFrame(pd.read_csv("special_measures_dimension.csv"))

cases = pd.read_csv("patients.csv")   
cases_df = pd.DataFrame(cases)    

cases_df

,patient_key,gender,age_group,acquisition_group,outbreak_related,outcome,reporting_phu,reporting_phu_city,onset_date,reported_date,test_date,specimen_date
0,1,FEMALE,50s,OB,y,Resolved,Peel Public Health,Mississauga,2020-05-12,2020-05-23,2020-12-03,2020-05-12
1,2,FEMALE,20s,CC,n,Resolved,Peel Public Health,Mississauga,2020-03-11,2020-12-05,2020-12-05,2020-09-29
2,3,MALE,50s,NO KNOWN EPI LINK,n,Resolved,York Region Public Health Services,Newmarket,2020-04-29,2020-10-06,2020-10-06,2020-08-21
3,4,FEMALE,30s,CC,n,Resolved,York Region Public Health Services,Newmarket,2020-03-23,2020-12-12,2020-12-12,2020-12-11
4,5,FEMALE,20s,CC,n,Resolved,York Region Public Health Services,Newmarket,2020-04-19,2020-04-19,2020-12-15,2020-12-13
...,...,...,...,...,...,...,...,...,...,...,...,...
97617,97618,MALE,20s,CC,y,Resolved,Peel Public Health,Mississauga,2020-12-05,2020-12-11,2020-12-11,2020-12-10
97618,97619,FEMALE,70s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-11-17,2020-11-20,2020-11-20,2020-11-18
97619,97620,FEMALE,20s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-10-19,2020-10-23,2020-10-23,2020-10-22
97620,97621,FEMALE,80s,OB,y,Fatal,York Region Public Health Services,Newmarket,2020-10-20,2020-10-23,2020-10-23,2020-10-20


In [3]:
#Fact table rows
fact_table_final =  pd.DataFrame(columns=['onset_date_surrogate','reported_date_surrogate','test_date_surrogate','specimen_date_surrogate', 'weather_surrogate','patient_surrogate', 'phu_location_surrogate', 'mobility_surrogate','special_measures_surrogate',"is_fatal", "is_unresolved", "is_resolved"])

fact_table_final

,onset_date_surrogate,reported_date_surrogate,test_date_surrogate,specimen_date_surrogate,weather_surrogate,patient_surrogate,phu_location_surrogate,mobility_surrogate,special_measures_surrogate,is_fatal,is_unresolved,is_resolved


In [15]:
#remove cases and drop duplicates for the cases data 
cases_df = cases_df[(cases_df.test_date >= '2020-09-01') & (cases_df.test_date <= '2020-12-31')]#TODO

cases_df

,patient_key,gender,age_group,acquisition_group,outbreak_related,outcome,reporting_phu,reporting_phu_city,onset_date,reported_date,test_date,specimen_date
0,1,FEMALE,50s,OB,y,Resolved,Peel Public Health,Mississauga,2020-05-12,2020-05-23,2020-12-03,2020-05-12
1,2,FEMALE,20s,CC,n,Resolved,Peel Public Health,Mississauga,2020-03-11,2020-12-05,2020-12-05,2020-09-29
2,3,MALE,50s,NO KNOWN EPI LINK,n,Resolved,York Region Public Health Services,Newmarket,2020-04-29,2020-10-06,2020-10-06,2020-08-21
3,4,FEMALE,30s,CC,n,Resolved,York Region Public Health Services,Newmarket,2020-03-23,2020-12-12,2020-12-12,2020-12-11
4,5,FEMALE,20s,CC,n,Resolved,York Region Public Health Services,Newmarket,2020-04-19,2020-04-19,2020-12-15,2020-12-13
...,...,...,...,...,...,...,...,...,...,...,...,...
97617,97618,MALE,20s,CC,y,Resolved,Peel Public Health,Mississauga,2020-12-05,2020-12-11,2020-12-11,2020-12-10
97618,97619,FEMALE,70s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-11-17,2020-11-20,2020-11-20,2020-11-18
97619,97620,FEMALE,20s,NO KNOWN EPI LINK,n,Resolved,Peel Public Health,Mississauga,2020-10-19,2020-10-23,2020-10-23,2020-10-22
97620,97621,FEMALE,80s,OB,y,Fatal,York Region Public Health Services,Newmarket,2020-10-20,2020-10-23,2020-10-23,2020-10-20


In [16]:
#drop duplicates
patients_data = cases_df.drop_duplicates(subset =['gender', 'age_group', 'acquisition_group', 'outbreak_related']) 
patients_data

,patient_key,gender,age_group,acquisition_group,outbreak_related,outcome,reporting_phu,reporting_phu_city,onset_date,reported_date,test_date,specimen_date
0,1,FEMALE,50s,OB,y,Resolved,Peel Public Health,Mississauga,2020-05-12,2020-05-23,2020-12-03,2020-05-12
1,2,FEMALE,20s,CC,n,Resolved,Peel Public Health,Mississauga,2020-03-11,2020-12-05,2020-12-05,2020-09-29
2,3,MALE,50s,NO KNOWN EPI LINK,n,Resolved,York Region Public Health Services,Newmarket,2020-04-29,2020-10-06,2020-10-06,2020-08-21
3,4,FEMALE,30s,CC,n,Resolved,York Region Public Health Services,Newmarket,2020-03-23,2020-12-12,2020-12-12,2020-12-11
5,6,FEMALE,80s,OB,y,Resolved,York Region Public Health Services,Newmarket,2020-06-11,2020-05-25,2020-09-23,2020-06-11
...,...,...,...,...,...,...,...,...,...,...,...,...
89190,89191,GENDER DIVERSE,20s,OB,y,Resolved,Toronto Public Health,Toronto,2020-09-29,2020-10-01,2020-10-01,2020-09-29
89229,89230,GENDER DIVERSE,40s,MISSING INFORMATION,n,Resolved,Toronto Public Health,Toronto,2020-11-05,2020-11-06,2020-11-06,2020-11-05
89619,89620,GENDER DIVERSE,<20,MISSING INFORMATION,y,Resolved,Toronto Public Health,Toronto,2020-11-17,2020-11-20,2020-11-20,2020-11-19
92164,92165,FEMALE,90+,OB,n,Resolved,York Region Public Health Services,Newmarket,2020-12-27,2020-12-28,2020-12-28,2020-12-27


In [17]:
#4 months 
start_date = datetime(2020, 9, 1).date()
end_date = datetime(2020, 12, 31).date()

#sort by reported_date
patients_data = patients_data.sort_values('reported_date')
patients_data.head()

regions = ["durham", "halton", "peel", "york", "toronto", "ottawa"]#list of regions

for idx, row in patients_data.iterrows():
    #reported date
    onset_date = row['onset_date']
    reported_date = row['reported_date']
    test_date = row['test_date']
    specimen_date = row['specimen_date']
    phu = row["reporting_phu"]
    
    #link dates key
    #onset_date key
    onset_key = dates_dimension[dates_dimension['date']==onset_date]['test_date_key'].values[0]

    #reported date key
    reported_key = dates_dimension[dates_dimension['date']== reported_date]['test_date_key'].values[0]

    #test date
    test_key = dates_dimension[dates_dimension['date']==test_date]['test_date_key'].values[0]

    #specimen date
    specimen_key = dates_dimension[dates_dimension['date']==specimen_date]['test_date_key'].values[0]

    #link regions
    for city in regions:
        if(city in phu.lower()):
            break#return x

    #link weather
    if(city == 'ottawa'):#Ottawa case
        weather_key = weather_dimension[(weather_dimension['station_name'] == 'OTTAWA') & (weather_dimension['date_col'] == test_date)]['weather_surrogate_key'].values[0] 
    else:#GTA case
        weather_key = weather_dimension[(weather_dimension['station_name'] == 'TORONTO') & (weather_dimension['date_col'] == test_date)]['weather_surrogate_key'].values[0] 


    #link mobility
    mobility_key = mobility_dimension[(mobility_dimension['date_col'] == test_date) & ( mobility_dimension['subregion'].str.contains(city, case=False))]['mobility_surrogate_key'].values[0] 


    #link measures
    measures_key = special_measures_dimension[((special_measures_dimension['start_date'] <= test_date) & (special_measures_dimension['end_date'] >= test_date)) & (special_measures_dimension['region'] == city) ]['special_measures_surrogate_key'].values[0]


    #add data to row
    fact_row = []#surrogate key
    fact_row.append(onset_key)#date
    fact_row.append(reported_key)#date 
    fact_row.append(test_key)#date 
    fact_row.append(specimen_key)#date 

    fact_row.append(weather_key)#weather
    fact_row.append(row['patient_key'])#patient_surrogate 
    fact_row.append(phu)#(phu_row["phu_location_key"])#phu TODO
    fact_row.append(mobility_key)#mobility
    fact_row.append(measures_key)#measures

    if(row["outcome"].lower() == "fatal"):
        fact_row.append(True)#is_fatal
        fact_row.append(False)#is_unresolved
        fact_row.append(True)#is_resolved
    elif(row["outcome"].lower() == "not resolved"):
        fact_row.append(False)#is_fatal
        fact_row.append(True)#is_unresolved
        fact_row.append(False)#is_resolved
    elif(row["outcome"].lower() == "resolved"):
        fact_row.append(False)#is_fatal
        fact_row.append(False)#is_unresolved
        fact_row.append(True)#is_resolved

    #add row    
    fact_table_final.loc[len(fact_table_final)] = fact_row


IndexError: index 0 is out of bounds for axis 0 with size 0

In [10]:
fact_table_final

,onset_date_surrogate,reported_date_surrogate,test_date_surrogate,specimen_date_surrogate,weather_surrogate,patient_surrogate,phu_location_surrogate,mobility_surrogate,special_measures_surrogate,is_fatal,is_unresolved,is_resolved
0,1,2,2,1,545,508,Peel Public Health,48065,102,False,False,True
1,1,2,2,1,545,611,Peel Public Health,48065,102,False,False,True
2,1,2,2,1,545,455,Peel Public Health,48065,102,False,False,True
3,2,3,3,2,546,674,Peel Public Health,48066,102,False,False,True
4,2,3,3,2,546,576,Peel Public Health,48066,102,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
459,114,119,119,118,662,51555,Toronto Public Health,50916,172,False,False,True
460,117,119,119,117,662,54661,Toronto Public Health,50916,172,False,False,True
461,118,119,119,118,662,92165,York Region Public Health Services,48543,188,False,False,True
462,117,121,121,118,664,54761,Toronto Public Health,50918,172,False,False,True


In [8]:
fact_table_final.fillna(0, inplace = True)

fact_table_final.to_csv("fact_table_final.csv",index=False)